# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [4]:
# Load the dataset stored in the file 'homepage_actions.csv' into a dataframe
import pandas as pd
df = pd.read_csv('homepage_actions.csv')

# Conduct exploratory analysis to get familiar with the data
# Investigate the id column
# How many viewers also clicked?
viewers_clicked = df[df['action'] == 'click']['id'].nunique()

# Are there any anomalies with the data; did anyone click who didn't view?
click_without_view = df[(df['action'] == 'click') & ~(df['id'].isin(df[df['action'] == 'view']['id']))]

# Is there any overlap between the control and experiment groups?
overlap = df.groupby('group')['id'].nunique()

# Print the results
viewers_clicked, click_without_view, overlap

(1860,
 Empty DataFrame
 Columns: [timestamp, id, group, action]
 Index: [],
 group
 control       3332
 experiment    2996
 Name: id, dtype: int64)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Explore the data
df.head()

# Investigate the id column
# How many viewers also clicked?
num_viewers_clicked = df[df['action'] == 'click']['id'].nunique()

# Are there any anomalies with the data; did anyone click who didn't view?
anomalies = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id']).sum()

# Is there any overlap between the control and experiment groups?
overlap = df[df['group'] == 'control']['id'].isin(df[df['group'] == 'experiment']['id']).sum()

# Print the results
num_viewers_clicked, anomalies, overlap

(1860, 1860, 0)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
# Convert the 'action' column to numeric
df['action'] = df['action'].map({'view': 0, 'click': 1})

# Step 1: Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

# Calculate the click-through rate for the control group
control_click_through_rate = df[df['group'] == 'control']['action'].mean()

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = control_click_through_rate * len(df[df['group'] == 'experiment'])

# Step 2: Calculate the number of standard deviations that the actual number of clicks was from this estimate.

# Calculate the actual number of clicks for the experiment group
actual_clicks_experiment = len(df[(df['group'] == 'experiment') & (df['action'] == 1)])

# Calculate the standard deviation for the number of successes in a binomial variable
variance = len(df[df['group'] == 'experiment']) * control_click_through_rate * (1 - control_click_through_rate)
std_dev = variance ** 0.5

# Calculate the number of standard deviations
num_std_devs = (actual_clicks_experiment - expected_clicks_experiment) / std_dev

# Step 3: Calculate the p-value using the normal distribution based on the z-score.

import scipy.stats as stats

# Calculate the p-value
p_value = 1 - stats.norm.cdf(num_std_devs)

expected_clicks_experiment, num_std_devs, p_value

(857.6848030018762, 2.71607302278784, 0.0033030672759265522)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [9]:
# Step 2: Calculate the number of standard deviations that the actual number of clicks was from this estimate.

# Calculate the mean and standard deviation of the "action" column in the df dataframe
mean_action = df["action"].mean()
std_action = df["action"].std()

# Calculate the actual number of clicks
actual_clicks = df[df["group"] == "click"]["action"].sum()

# Calculate the estimated number of clicks based on the mean
estimated_clicks = mean_action * len(df[df["group"] == "click"])

# Calculate the number of standard deviations that the actual number of clicks was from the estimate
num_std_deviations = (actual_clicks - estimated_clicks) / std_action

num_std_deviations

0.0

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
# Step 3: Calculate a p-value using the normal distribution based on the z-score

import scipy.stats as stats

# Calculate the p-value using the cumulative distribution function (CDF) of the standard normal distribution
p_value = 1 - stats.norm.cdf(num_std_deviations)

p_value

0.5

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.